<a href="https://colab.research.google.com/github/danie-cp/Projects/blob/main/DSC326Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/DSC326_Spring2025/PythonFolder

/content/drive/MyDrive/Colab Notebooks/DSC326_Spring2025/PythonFolder


#Part 1

(1)

Kaggle: https://www.kaggle.com/datasets/iammustafatz/diabetes-prediction-dataset

The Diabetes Predicion Database consists of 100,000 patient medical history and diabetic status. The country of origin and year are unknown due to privacy reasons. The dataset consists of 2 nominal categorical variables, *gender* and *smoking* *history*; 2 binary features *hypertension* (hight blood pressure) and *heart diseases* presence; in addition has 3 numerical variables; *BMI index*, *HbA1c* (average blood sugar level over the past 2 to 3 months) and *Blood Glucose level*. These features can be used to classify whether a patient is at risk of developing diabetes, the target feature. The goal at hand is to  classify whether a patient has dieabetes and can help locate which patients are at risk and may need preventive treatment.
There is a balanced set of both numeric and categorical features.


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, OneHotEncoder
from itertools import product
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import uniform

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import metrics

(2)

In [ ]:
diabetes = pd.read_csv("diabetes_prediction_dataset.csv")
diabetes.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [ ]:
X = diabetes.drop("diabetes", axis=1) # Features
y = diabetes["diabetes"] # Target variable

In [ ]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23, stratify=y)

(3)

In [ ]:
# Identify categorical and numerical columns
categorical_features = ["gender", "smoking_history"]
numerical_features = ["age","bmi", "HbA1c_level", "blood_glucose_level"]
binary_features = ["hypertension","heart_disease"]

In [ ]:
# One-hot encode categorical variables using training data only
encoder = OneHotEncoder(drop = "first", handle_unknown="ignore")
X_train_categorical = encoder.fit_transform(X_train[categorical_features]).toarray()

# Standardize numerical features using training data only
scaler = StandardScaler()                  #subtract mean by the sigma
X_train_numerical = scaler.fit_transform(X_train[numerical_features])

X_train_binary = X_train[binary_features]
X_train_binary = X_train_binary.values
# Concatenate transformed numerical and categorical features
X_train_transformed = np.hstack((X_train_numerical, X_train_binary, X_train_categorical))

In [ ]:
# One-hot encode y-test variables (used later for training on test set)
X_test_categorical = encoder.transform(X_test[categorical_features]).toarray()

# Standardize numerical features using training data only
X_test_numerical = scaler.transform(X_test[numerical_features])

X_test_binary = X_test[binary_features]
X_test_binary = X_test_binary.values

# Concatenate transformed numerical and categorical features
X_test_transformed = np.hstack((X_test_numerical,X_test_binary, X_test_categorical))

##K-Nearest Neigbors

(4)

In [ ]:
# Perform 5-fold cross-validation to select the best k
k_values = range(1, 20)
accuracy_scores = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train_transformed, y_train, cv=5, scoring='accuracy')
    overall_accuracy = np.mean(scores)  # Compute mean accuracy
    accuracy_scores.append(overall_accuracy)

In [ ]:
# Find the best k with the highest accuracy
best_k = k_values[np.argmax(accuracy_scores)]
best_accuracy = max(accuracy_scores)
accuracy_scores

[np.float64(0.947775),
 np.float64(0.9602999999999999),
 np.float64(0.957225),
 np.float64(0.9606),
 np.float64(0.960125),
 np.float64(0.9610749999999999),
 np.float64(0.9611375000000001),
 np.float64(0.9611374999999999),
 np.float64(0.9612124999999999),
 np.float64(0.9611875),
 np.float64(0.961525),
 np.float64(0.961375),
 np.float64(0.9616374999999999),
 np.float64(0.9613999999999999),
 np.float64(0.9615375),
 np.float64(0.9613125),
 np.float64(0.9614749999999999),
 np.float64(0.9611625),
 np.float64(0.9614)]

In [ ]:
# Print results
print(f"Best k: {best_k}")

Best k: 13


In [ ]:
# Train and evaluate the model with the best k and calculate accuracy on the training set
knn_best = KNeighborsClassifier(n_neighbors=best_k)
knn_best.fit(X_train_transformed, y_train)

y_train_pred = knn_best.predict(X_train_transformed)
K_NN_Train_ACC = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", K_NN_Train_ACC)

Training Accuracy: 0.964475


In [ ]:
# calculate accuracy on the test set
y_test_pred = knn_best.predict(X_test_transformed)
K_NN_Test_ACC = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", K_NN_Test_ACC)

Test Accuracy: 0.96365


##Multilayer Perceptron Classifier

(4)

In [ ]:
# Define hyperparameter grid
hidden_layers_options = [1, 2]
hidden_nodes_options = [15,25,30]
activation_functions = ['logistic', 'relu']   #tanh was also tested

# Store results
results = []

# Combinations of hyperparameters
prodvalues = product(hidden_layers_options, hidden_nodes_options, activation_functions)
print(list(prodvalues))

[(1, 15, 'logistic'), (1, 15, 'relu'), (1, 25, 'logistic'), (1, 25, 'relu'), (1, 30, 'logistic'), (1, 30, 'relu'), (2, 15, 'logistic'), (2, 15, 'relu'), (2, 25, 'logistic'), (2, 25, 'relu'), (2, 30, 'logistic'), (2, 30, 'relu')]


In [ ]:
# Perform 5-fold cross-validation for each combination of hyperparameters

for hidden_layer, hidden_node, activation_function in product(hidden_layers_options, hidden_nodes_options, activation_functions):
    mlp = MLPClassifier(hidden_layer_sizes=(hidden_node,)*hidden_layer, activation=activation_function, max_iter=1000, random_state=23)
    accuracy = cross_val_score(mlp, X_train_transformed, y_train, cv=5, scoring='accuracy')
    mean_accuracy = accuracy.mean()
    results.append((hidden_layer, hidden_node, activation_function, mean_accuracy))
    print(f"Hidden Layers: {hidden_layer}, Hidden Nodes: {hidden_node}, Activation Function: {activation_function}, Accuracy: {mean_accuracy}")

Hidden Layers: 1, Hidden Nodes: 15, Activation Function: logistic, Accuracy: 0.9611124999999999
Hidden Layers: 1, Hidden Nodes: 15, Activation Function: relu, Accuracy: 0.9711624999999999
Hidden Layers: 1, Hidden Nodes: 25, Activation Function: logistic, Accuracy: 0.9610750000000001
Hidden Layers: 1, Hidden Nodes: 25, Activation Function: relu, Accuracy: 0.9715374999999999
Hidden Layers: 1, Hidden Nodes: 30, Activation Function: logistic, Accuracy: 0.9609875000000001
Hidden Layers: 1, Hidden Nodes: 30, Activation Function: relu, Accuracy: 0.9715499999999999
Hidden Layers: 2, Hidden Nodes: 15, Activation Function: logistic, Accuracy: 0.9713875
Hidden Layers: 2, Hidden Nodes: 15, Activation Function: relu, Accuracy: 0.9714500000000001
Hidden Layers: 2, Hidden Nodes: 25, Activation Function: logistic, Accuracy: 0.9708375
Hidden Layers: 2, Hidden Nodes: 25, Activation Function: relu, Accuracy: 0.9712249999999999
Hidden Layers: 2, Hidden Nodes: 30, Activation Function: logistic, Accuracy: 0

In [ ]:
# Select the best hyperparameter setting
best_hidden_layers, best_hidden_nodes, best_activation_function, best_accuracy = max(results, key=lambda x: x[3])
print(best_hidden_layers, best_hidden_nodes, best_activation_function, best_accuracy)

1 30 relu 0.9715499999999999


(5)

In [ ]:
# Train and evaluate the model with the best k and calculate accuracy on the training set
final_MLP_model = MLPClassifier(hidden_layer_sizes=(best_hidden_nodes,)*best_hidden_layers, activation=best_activation_function, max_iter=1000, random_state=23)
final_MLP_model.fit(X_train_transformed, y_train)

# Predict on training set
y_train_pred = final_MLP_model.predict(X_train_transformed)

# Calculate accuracy on the training set
MLP_Train_ACC = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", MLP_Train_ACC)

Training Accuracy: 0.9721125


In [ ]:
# Predict on test set
y_test_pred = final_MLP_model.predict(X_test_transformed)

# Calculate accuracy on test set
MLP_Test_ACC  = accuracy_score(y_test, y_test_pred)

# Print accuracy score on test set
print("Test Accuracy:", MLP_Test_ACC )

Test Accuracy: 0.97325


##Random Forrest

(4)

In [ ]:
# Define hyperparameter grid
num_trees_options = [50,100,200]
max_depth_options = [5,10,20]
min_samples_leaf_options = [5, 10]

# Store results
results = []

# Combinations of hyperparameters
prodvalues = product(num_trees_options, max_depth_options, min_samples_leaf_options)
print(list(prodvalues))

[(50, 5, 5), (50, 5, 10), (50, 10, 5), (50, 10, 10), (50, 20, 5), (50, 20, 10), (100, 5, 5), (100, 5, 10), (100, 10, 5), (100, 10, 10), (100, 20, 5), (100, 20, 10), (200, 5, 5), (200, 5, 10), (200, 10, 5), (200, 10, 10), (200, 20, 5), (200, 20, 10)]


In [ ]:
# Perform 5-fold cross-validation for each combination of hyperparameters
for num_trees, max_depth, min_samples_leaf in product(num_trees_options, max_depth_options, min_samples_leaf_options):
    rf = RandomForestClassifier(n_estimators=num_trees, max_depth=max_depth, min_samples_leaf=min_samples_leaf, random_state=23)

    # Compute RMSE using cross-validation
    accuracy = cross_val_score(rf, X_train_transformed, y_train, cv=5, scoring='accuracy')
    mean_accuracy = accuracy.mean()
    results.append((num_trees, max_depth, min_samples_leaf, mean_accuracy))
    print(f"num_trees: {num_trees}, max_depth: {max_depth}, min_samples_leaf: {min_samples_leaf}, Accuracy: {mean_accuracy}")

num_trees: 50, max_depth: 5, min_samples_leaf: 5, Accuracy: 0.9715374999999999
num_trees: 50, max_depth: 5, min_samples_leaf: 10, Accuracy: 0.9715374999999999
num_trees: 50, max_depth: 10, min_samples_leaf: 5, Accuracy: 0.971525
num_trees: 50, max_depth: 10, min_samples_leaf: 10, Accuracy: 0.9715250000000001
num_trees: 50, max_depth: 20, min_samples_leaf: 5, Accuracy: 0.9717125
num_trees: 50, max_depth: 20, min_samples_leaf: 10, Accuracy: 0.97165
num_trees: 100, max_depth: 5, min_samples_leaf: 5, Accuracy: 0.9715374999999999
num_trees: 100, max_depth: 5, min_samples_leaf: 10, Accuracy: 0.9715374999999999
num_trees: 100, max_depth: 10, min_samples_leaf: 5, Accuracy: 0.9715625000000001
num_trees: 100, max_depth: 10, min_samples_leaf: 10, Accuracy: 0.9715250000000001
num_trees: 100, max_depth: 20, min_samples_leaf: 5, Accuracy: 0.9717375
num_trees: 100, max_depth: 20, min_samples_leaf: 10, Accuracy: 0.9715999999999999
num_trees: 200, max_depth: 5, min_samples_leaf: 5, Accuracy: 0.97153749

In [ ]:
# Select the best hyperparameter setting
best_num_trees, best_max_depth, best_min_samples_leaf, best_accuracy = max(results, key=lambda x: x[3])
print(best_num_trees, best_max_depth, best_min_samples_leaf, best_accuracy)

100 20 5 0.9717375


(5)

In [ ]:
# Train and evaluate the model with the best k and calculate accuracy on the training set
final_rf_model = RandomForestClassifier(n_estimators=best_num_trees, max_depth=best_max_depth, min_samples_leaf=best_min_samples_leaf, random_state=23)
final_rf_model.fit(X_train_transformed, y_train)

# Predict on training set
y_train_pred = final_rf_model.predict(X_train_transformed)

# Calculate accuracy on the training set
RF_Train_ACC = accuracy_score(y_train, y_train_pred)
print("Training Accuracy", RF_Train_ACC)

Training Accuracy 0.97325


In [ ]:
# Predict on test set
y_test_pred = final_rf_model.predict(X_test_transformed)

# Calculate accuracy on test set
RF_Test_ACC = accuracy_score(y_test, y_test_pred)

# Print accuracy score on test set
print("Test Accuracy", RF_Test_ACC)

Test Accuracy 0.9732


##Support Vector Classifier

(4)

In [ ]:
#Define hyperparameter grid
kernel_func_options = ['linear', 'poly', 'rbf']
C_val_options = [0.1,1,10,100]

#Store results
results = []

#Combinations of hyperparameters
prodvalues = product(kernel_func_options, C_val_options)
print(list(prodvalues))

[('linear', 0.1), ('linear', 1), ('linear', 10), ('linear', 100), ('poly', 0.1), ('poly', 1), ('poly', 10), ('poly', 100), ('rbf', 0.1), ('rbf', 1), ('rbf', 10), ('rbf', 100)]


In [ ]:
# Perform 5-fold cross-validation for each combination of hyperparameters
for kernel_func, C_val in product(kernel_func_options, C_val_options):
    svc_model = SVC(kernel=kernel_func, C=C_val, random_state=23)

    # Compute accuracy using 5-fold cross-validation on the training set
    cv_scores = cross_val_score(svc_model, X_train_transformed, y_train, cv=5, scoring='accuracy')

    # Compute overall accuracy from cross-validation
    mean_accuracy = np.mean(cv_scores)

    results.append((kernel_func, C_val, mean_accuracy))

    # print kernel_func, C_val and mean_accuracy
    print(f'kernel = {kernel_func}, C = {C_val}: mean accuracy = {mean_accuracy}')

kernel = linear, C = 0.1: mean accuracy = 0.9609500000000001
kernel = linear, C = 1: mean accuracy = 0.9610749999999999
kernel = linear, C = 10: mean accuracy = 0.96105
kernel = linear, C = 100: mean accuracy = 0.9610625
kernel = poly, C = 0.1: mean accuracy = 0.9638625000000001
kernel = poly, C = 1: mean accuracy = 0.9651
kernel = poly, C = 10: mean accuracy = 0.967375
kernel = poly, C = 100: mean accuracy = 0.96835
kernel = rbf, C = 0.1: mean accuracy = 0.9630124999999999
kernel = rbf, C = 1: mean accuracy = 0.96545
kernel = rbf, C = 10: mean accuracy = 0.9685750000000001
kernel = rbf, C = 100: mean accuracy = 0.9692124999999999


In [ ]:
# Select the best hyperparameter setting
best_kernel_func, best_C_val, best_acc = max(results, key=lambda x: x[2])
print(best_kernel_func, best_C_val, best_acc)

rbf 100 0.9692124999999999


(5)

In [ ]:
# Train and evaluate the model and calculate accuracy on the training set

svc_model = SVC(kernel=best_kernel_func, C=best_C_val, random_state=23)
svc_model.fit(X_train_transformed, y_train)

# Predict on training set
y_train_pred = svc_model.predict(X_train_transformed)

# Calculate accuracy on the training set
y_train_pred = svc_model.predict(X_train_transformed)
SVC_Train_ACC = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", SVC_Train_ACC)

Training Accuracy: 0.97285


In [ ]:
# calculate accuracy on the test set
y_test_pred = svc_model.predict(X_test_transformed)
SVC_Test_ACC = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", SVC_Test_ACC)

Test Accuracy: 0.97165


##Logistic Regression

In [ ]:
# initialize the model
log_regression = LogisticRegression(penalty = None)  # no regularization

In [ ]:
#using sklearn

# fit the model using the training data
log_regression.fit(X_train_transformed, y_train)

# use model to make predictions on the training data
y_train_pred = log_regression.predict(X_train_transformed)
# accuracy on the training data
LR_Train_ACC = metrics.accuracy_score(y_train, y_train_pred)
print('accuracy on train set:', LR_Train_ACC)

accuracy on train set: 0.960475


In [ ]:
# use model to make predictions on the test data
y_test_pred2 = log_regression.predict(X_test_transformed)
# accuracy on the test data
LR_Test_ACC = metrics.accuracy_score(y_test, y_test_pred2)
print('accuracy on test set:', LR_Test_ACC)

accuracy on test set: 0.9604


##(5)

Native Algorithm

In [ ]:
y_test_freq = pd.DataFrame(y_train)
freq_table = y_train.value_counts()
print(freq_table)

size = len(y_test)
most_freq_arr = np.zeros(size)
Native_Test_ACC = accuracy_score(y_test, most_freq_arr)
print("Test Accuracy:",Native_Test_ACC)

diabetes
0    73200
1     6800
Name: count, dtype: int64
Test Accuracy: 0.915


Results

In [ ]:
#initialize the Result Table
data = {'Algorithm': ['K-NN','MLP','Random Forest','SVC','Logistic Regression','Native'],
        'Mean Accuracy on Test Set': [K_NN_Test_ACC, MLP_Test_ACC,RF_Test_ACC,SVC_Test_ACC,LR_Test_ACC,Native_Test_ACC],
        'Mean Accuracy on Train Set': [K_NN_Train_ACC, MLP_Train_ACC,RF_Train_ACC,SVC_Train_ACC,LR_Train_ACC,0]}

df = pd.DataFrame(data)

#Sort Accuracy
df_sorted = df.sort_values(by='Mean Accuracy on Test Set',ascending=False)
df_sorted

,Algorithm,Mean Accuracy on Test Set,Mean Accuracy on Train Set
1,MLP,0.97325,0.972113
2,Random Forest,0.97320,0.973250
3,SVC,0.97165,0.972850
0,K-NN,0.96365,0.964475
4,Logistic Regression,0.96040,0.960475
5,Native,0.91500,0.000000


Overall, MLP had the highest test set accuracy of .97265. and Logisitic Regression performed the lowest with an accuracy of .91500

 Compared to the Native Algorithm, all of the Models performed sustainably better.

##(6)

Interestingly, the dataset trained slowly in tanh compared to logistic and relu. And logisitc trained better with higher nodes while rulu performed better with lower hidden nodes. In addition, K-NN performed better with a lower k value compared to a high one. MLP performed better with relu for ranges 15-30.I also found it interesting that this time around the native algorithm did not perform slighlty worse but the other models performed sustainably better.



#Part 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/DSC326_Spring2025/PythonFolder

/content/drive/MyDrive/DSC326_Spring2025/PythonFolder


In [ ]:
pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error, r2_score
from itertools import product

from sklearn.neighbors import KNeighborsRegressor

from sklearn.preprocessing import OneHotEncoder
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import plot_tree
from sklearn.preprocessing import MinMaxScaler

from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

from scipy.stats import uniform
from skopt import BayesSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

from skopt.space import Real, Categorical


https://archive.ics.uci.edu/dataset/397/las+vegas+strip

Moro, S., Rita, P., & Coelho, J. (2017). Las Vegas Strip [Dataset]. UCI Machine Learning Repository. https://doi.org/10.24432/C5QG7W.



The dataset consists of 504 hotel reviews, all collected between January and August of 2015. These reviews represent 24 entries per hotel (two per month), and span a total of 21 different hotels in Las Vegas. Each record includes information about the reviewer, hotel-specific features, and review data. The data was probabiliy collected to support research on travel behavior and hotel selection, possibly to improve marketing strategies or user recommendation systems on trip planning tplatforms

The goal of the regression problem is to predict Member years, the number of years a user has been active on TripAdvisor, based on 20 tuned features. These include numeric variables such as Nr. reviews, Nr. hotel reviews, Helpful votes, and Nr. rooms; binary features such as Pool, Gym, Spa, Casino, and Free internet; ordinal categorical features like Hotel stars and Score; and nominal categorical features like Traveler type, User country, User continent, Hotel name, and Period of stay. By modeling Member years, we aim to understand the characteristics of more experienced users and how different factors contribute to user engagement on review platforms.

In [ ]:
df = pd.read_csv('LasVegasTripAdvisorReviews-Dataset.csv', delimiter=';')
df.head()

,User country,Nr. reviews,Nr. hotel reviews,Helpful votes,Score,Period of stay,Traveler type,Pool,Gym,Tennis court,Spa,Casino,Free internet,Hotel name,Hotel stars,Nr. rooms,User continent,Member years,Review month,Review weekday
0,USA,11,4,13,5,Dec-Feb,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,9,January,Thursday
1,USA,119,21,75,3,Dec-Feb,Business,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,3,January,Friday
2,USA,36,9,25,5,Mar-May,Families,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,2,February,Saturday
3,UK,14,7,14,4,Mar-May,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,Europe,6,February,Friday
4,Canada,5,5,2,4,Mar-May,Solo,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,7,March,Tuesday


In [ ]:
df['Hotel stars'] = df['Hotel stars'].astype(str).str.replace(',', '.').astype(float)
df['Member years'] = df['Member years'].clip(lower=0)

In [ ]:
df['Hotel stars'] = df['Hotel stars'].astype(str).str.replace(',', '.').astype(float)
df['Member years'] = df['Member years'].clip(lower=0)

X = df.drop(['Member years', 'Review month', 'Review weekday'], axis=1)
y = df['Member years']

In [ ]:
nominal_cols = ['Traveler type', 'User continent', 'Hotel name', 'User country', 'Period of stay']
ordinal_cols = ['Hotel stars', 'Score']
numeric_cols = ['Nr. rooms', 'Nr. reviews', 'Nr. hotel reviews', 'Helpful votes']
binary_features = ['Pool', 'Gym', 'Tennis court', 'Spa', 'Casino', 'Free internet']

X[binary_features] = X[binary_features].apply(lambda x: x.map({'YES': 1, 'NO': 0}))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23)

In [ ]:
ohe = OneHotEncoder(drop='first', handle_unknown='ignore')
X_train_nominal = pd.DataFrame(ohe.fit_transform(X_train[nominal_cols]).toarray(),
                               columns=ohe.get_feature_names_out(nominal_cols),
                               index=X_train.index)
X_test_nominal = pd.DataFrame(ohe.transform(X_test[nominal_cols]).toarray(),
                              columns=ohe.get_feature_names_out(nominal_cols),
                              index=X_test.index)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [ ]:
hotel_stars_map = [1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
score_map = [1, 2, 3, 4, 5]
ordinal_map = [hotel_stars_map, score_map]

ordinal_encoder = OrdinalEncoder(categories=ordinal_map)
X_train_ordinal = pd.DataFrame(ordinal_encoder.fit_transform(X_train[ordinal_cols]),
                               columns=ordinal_cols,
                               index=X_train.index)
X_test_ordinal = pd.DataFrame(ordinal_encoder.transform(X_test[ordinal_cols]),
                              columns=ordinal_cols,
                              index=X_test.index)

In [ ]:
scaler = StandardScaler()
X_train_numeric = pd.DataFrame(scaler.fit_transform(X_train[numeric_cols]),
                               columns=numeric_cols,
                               index=X_train.index)
X_test_numeric = pd.DataFrame(scaler.transform(X_test[numeric_cols]),
                              columns=numeric_cols,
                              index=X_test.index)


In [ ]:
X_train_processed = pd.concat([X_train_ordinal, X_train_numeric, X_train_nominal], axis=1)
X_test_processed = pd.concat([X_test_ordinal, X_test_numeric, X_test_nominal], axis=1)

X_train_processed = X_train_processed.fillna(X_train_processed.mean())
X_test_processed = X_test_processed.fillna(X_test_processed.mean())

In [ ]:
print(X_train['Hotel stars'].unique())

[3.5 4.  5.  3.  4.5]


# KNN

In [ ]:
k_values = range(1, 101)
rmse_scores = []

for k in k_values:
    knn = KNeighborsRegressor(n_neighbors=k)
    scores = cross_val_score(knn, X_train_processed, y_train, cv=5, scoring='neg_root_mean_squared_error')
    rmse_scores.append(-np.mean(scores))

In [ ]:
best_k = k_values[np.argmin(rmse_scores)]
best_rmse = min(rmse_scores)

knn_best = KNeighborsRegressor(n_neighbors=best_k)
knn_best.fit(X_train_processed, y_train)

y_train_pred = knn_best.predict(X_train_processed)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

y_test_pred = knn_best.predict(X_test_processed)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"Best k: {best_k}")
print(f"Cross-validated RMSE: {best_rmse:.4f}")
print(f"Train RMSE: {train_rmse:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")

Best k: 43
Cross-validated RMSE: 2.9063
Train RMSE: 2.8381
Test RMSE: 2.7856


# MPL

In [ ]:
# Define hyperparameter grid
hidden_layer_options = [1, 2]
hidden_nodes_options = [5, 10, 20]
activation_functions = ['logistic', 'relu']

# Store results
results = []

In [ ]:
# Combinations of hyperparameters
prodvalues = product(hidden_layer_options, hidden_nodes_options, activation_functions)
print(list(prodvalues))

[(1, 5, 'logistic'), (1, 5, 'relu'), (1, 10, 'logistic'), (1, 10, 'relu'), (1, 20, 'logistic'), (1, 20, 'relu'), (2, 5, 'logistic'), (2, 5, 'relu'), (2, 10, 'logistic'), (2, 10, 'relu'), (2, 20, 'logistic'), (2, 20, 'relu')]


In [ ]:
for layers, nodes, activation in product(hidden_layer_options, hidden_nodes_options, activation_functions):
    hidden_layer_sizes = tuple([nodes] * layers)
    mlp = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes, activation=activation,
                       random_state=42, max_iter=1000)

    # 5-fold cross-validation RMSE
    scores = cross_val_score(mlp, X_train_processed, y_train, cv=5, scoring='neg_root_mean_squared_error')
    mean_rmse = -np.mean(scores)

    results.append((hidden_layer_sizes, activation, mean_rmse))
    print(f"Layers: {layers}, Nodes: {nodes}, Activation: {activation} => RMSE: {mean_rmse:.4f}")

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perce

Layers: 1, Nodes: 5, Activation: logistic => RMSE: 2.8431


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perce

Layers: 1, Nodes: 5, Activation: relu => RMSE: 2.9765


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perce

...

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perce

Layers: 2, Nodes: 20, Activation: logistic => RMSE: 2.9330


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Layers: 2, Nodes: 20, Activation: relu => RMSE: 3.9156


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
# understand structure of above for loop
for layers, nodes, activation in product(hidden_layer_options, hidden_nodes_options, activation_functions):
    print("layers, nodes, activation:", layers, nodes, activation)
    hidden_layer_sizes = tuple([nodes] * layers)
    print(hidden_layer_sizes)

layers, nodes, activation: 1 5 logistic
(5,)
layers, nodes, activation: 1 5 relu
(5,)
layers, nodes, activation: 1 10 logistic
(10,)
layers, nodes, activation: 1 10 relu
(10,)
layers, nodes, activation: 1 20 logistic
(20,)
layers, nodes, activation: 1 20 relu
(20,)
layers, nodes, activation: 2 5 logistic
(5, 5)
layers, nodes, activation: 2 5 relu
(5, 5)
layers, nodes, activation: 2 10 logistic
(10, 10)
layers, nodes, activation: 2 10 relu
(10, 10)
layers, nodes, activation: 2 20 logistic
(20, 20)
layers, nodes, activation: 2 20 relu
(20, 20)


In [ ]:
# Select the best hyperparameter setting
best_hidden_layer_sizes, best_activation, best_rmse = min(results, key=lambda x: x[2])
print(f"Best hidden layer sizes: {best_hidden_layer_sizes}, Best activation: {best_activation}, CV RMSE: {best_rmse}")

# Train final model on the full training set with the best hyperparameters
final_model = MLPRegressor(hidden_layer_sizes=best_hidden_layer_sizes,
                            activation=best_activation,
                            random_state=42,
                            max_iter=1000)

# Fit the model on the training data
final_model.fit(X_train_processed, y_train)  # Use processed X_train from preprocessor

# Evaluate on the training set (we will calculate RMSE on the training set as well)
y_train_pred = final_model.predict(X_train_processed)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

# Evaluate on the test set
y_test_pred = final_model.predict(X_test_processed)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

# Print results
print(f"Training RMSE: {train_rmse:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")

Best hidden layer sizes: (10,), Best activation: logistic, CV RMSE: 2.8381861798814434
Training RMSE: 2.5984
Test RMSE: 2.6090


In [ ]:
# Convert results to DataFrame for better visualization
results_df = pd.DataFrame(results, columns=["Hidden Layer Sizes", "Activation Function", "Cross-Validation RMSE"])
results_df

,Hidden Layer Sizes,Activation Function,Cross-Validation RMSE
0,"(5,)",logistic,2.843056
1,"(5,)",relu,2.976501
2,"(10,)",logistic,2.838186
3,"(10,)",relu,2.991443
4,"(20,)",logistic,2.881807
5,"(20,)",relu,3.078038
6,"(5, 5)",logistic,2.848048
7,"(5, 5)",relu,3.160639
8,"(10, 10)",logistic,2.852548
9,"(10, 10)",relu,3.279257


# Random Forest

In [ ]:
randstate=23

In [ ]:
# Define hyperparameter grid
num_trees_options = [100, 200]
max_depth_options = [20]
min_samples_leaf_options = [5, 10]

# Store results
results = []

# Combinations of hyperparameters
prodvalues = product(num_trees_options, max_depth_options, min_samples_leaf_options)
print(list(prodvalues))

[(100, 20, 5), (100, 20, 10), (200, 20, 5), (200, 20, 10)]


In [ ]:
# Perform 5-fold cross-validation for each combination of hyperparameters
for num_trees, max_depth, min_samples_leaf in product(num_trees_options, max_depth_options, min_samples_leaf_options):
    rf_pruned_reg = RandomForestRegressor(n_estimators=num_trees, max_depth=max_depth, min_samples_leaf=min_samples_leaf,\
                                          random_state=randstate)

    # Compute RMSE using cross-validation
    rmse_scores = cross_val_score(rf_pruned_reg, X_train_processed, y_train, cv=5, scoring='neg_mean_squared_error')
    mean_rmse = np.sqrt(-np.mean(rmse_scores))  # Correct overall RMSE calculation

    results.append((num_trees, max_depth, min_samples_leaf, mean_rmse))
    print(f"num_trees: {num_trees}, max_depth: {max_depth}, min_samples_leaf: {min_samples_leaf}, RMSE: {mean_rmse}")

num_trees: 100, max_depth: 20, min_samples_leaf: 5, RMSE: 2.786163132108272
num_trees: 100, max_depth: 20, min_samples_leaf: 10, RMSE: 2.793160955683453
num_trees: 200, max_depth: 20, min_samples_leaf: 5, RMSE: 2.783144399641168
num_trees: 200, max_depth: 20, min_samples_leaf: 10, RMSE: 2.794449253915318


In [ ]:
results_df = pd.DataFrame(results, columns=["Number of Trees", "Max Depth", "Min Samples Leaf", "Cross-Validation RMSE"])
results_df

,Number of Trees,Max Depth,Min Samples Leaf,Cross-Validation RMSE
0,100,20,5,2.786163
1,100,20,10,2.793161
2,200,20,5,2.783144
3,200,20,10,2.794449


In [ ]:
# Select the best hyperparameter setting
best_num_trees, best_max_depth, best_min_samples_leaf, best_rmse = min(results, key=lambda x: x[3])
print(best_num_trees, best_max_depth, best_min_samples_leaf, best_rmse)

200 20 5 2.783144399641168


In [ ]:
# Train final model on the full training set with the best hyperparameters
final_model = RandomForestRegressor(n_estimators=best_num_trees, max_depth=best_max_depth, min_samples_leaf=best_min_samples_leaf,\
                                     random_state=randstate)
final_model.fit(X_train_processed, y_train)

RandomForestRegressor(max_depth=20, min_samples_leaf=5, n_estimators=200,
                      random_state=23)

In [ ]:
# Predict on training set
y_train_pred = final_model.predict(X_train_processed)

# Calculate RMSE on the training set
train_rmse = root_mean_squared_error(y_train, y_train_pred)

# Print RMSE score on training set
print(f"Training RMSE: {train_rmse:.4f}")

Training RMSE: 1.9240


In [ ]:
# Predict on test set
y_test_pred = final_model.predict(X_test_processed)

# Calculate RMSE on the test set
test_rmse = root_mean_squared_error(y_test, y_test_pred)

# Print RMSE score on test set
print(f"Test RMSE: {test_rmse:.4f}")

Test RMSE: 2.5552


# Support Vector Regressor

In [ ]:
svr = SVR()
param_grid = {
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 0.5],
    'kernel': ['linear', 'rbf', 'poly']
}

grid_search = GridSearchCV(
    estimator=svr,
    param_grid=param_grid,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1
)

grid_search.fit(X_train_processed, y_train)

GridSearchCV(cv=5, estimator=SVR(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100], 'epsilon': [0.01, 0.1, 0.5],
                         'kernel': ['linear', 'rbf', 'poly']},
             scoring='neg_root_mean_squared_error')

In [ ]:
best_svr = grid_search.best_estimator_

y_train_pred = best_svr.predict(X_train_processed)
y_test_pred = best_svr.predict(X_test_processed)

# RMSE
train_rmse = root_mean_squared_error(y_train, y_train_pred)
test_rmse = root_mean_squared_error(y_test, y_test_pred)

# Output
print("Best Hyperparameters:", grid_search.best_params_)
print(f"Train RMSE: {train_rmse:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")

Best Hyperparameters: {'C': 10, 'epsilon': 0.5, 'kernel': 'rbf'}
Train RMSE: 2.4300
Test RMSE: 2.6298


In [ ]:
# GridSearchCV
grid_svr = SVR()
param_grid = {
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 0.5],
    'kernel': ['linear', 'rbf', 'poly']
}
grid_search = GridSearchCV(grid_svr, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train_processed, y_train)
print("GridSearchCV:")
print("Train RMSE:", root_mean_squared_error(y_train, grid_search.predict(X_train_processed)))
print("Test RMSE:", root_mean_squared_error(y_test, grid_search.predict(X_test_processed)))
print("Best Params:", grid_search.best_params_)

GridSearchCV:
Train RMSE: 2.4300061559105095
Test RMSE: 2.629805770638538
Best Params: {'C': 10, 'epsilon': 0.5, 'kernel': 'rbf'}


In [ ]:
# RandomizedSearchCV
random_svr = SVR()
param_distributions = {
    'C': uniform(0.1, 100),
    'epsilon': uniform(0.01, 1),
    'kernel': ['linear', 'rbf', 'poly']
}
random_search = RandomizedSearchCV(random_svr, param_distributions, n_iter=30,
                                   cv=5, scoring='neg_mean_squared_error', n_jobs=-1, random_state=42)
random_search.fit(X_train_processed, y_train)
print("\nRandomizedSearchCV:")
print("Train RMSE:", root_mean_squared_error(y_train, random_search.predict(X_train_processed)))
print("Test RMSE:", root_mean_squared_error(y_test, random_search.predict(X_test_processed)))
print("Best Params:", random_search.best_params_)


RandomizedSearchCV:
Train RMSE: 2.5171240969650728
Test RMSE: 2.6330469457112575
Best Params: {'C': np.float64(5.741157902710025), 'epsilon': np.float64(0.7319987722668247), 'kernel': 'rbf'}


In [ ]:
# BayesSearchCV
search_space = {
    'C': Real(1e-2, 100, prior='log-uniform'),
    'epsilon': Real(1e-2, 1.0, prior='log-uniform'),
    'kernel': Categorical(['linear', 'rbf', 'poly'])
}
bayes_search = BayesSearchCV(SVR(), search_spaces=search_space, n_iter=30,
                             cv=5, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)
bayes_search.fit(X_train_processed, y_train)
print("\nBayesSearchCV:")
print("Train RMSE:", root_mean_squared_error(y_train, bayes_search.predict(X_train_processed)))
print("Test RMSE:", root_mean_squared_error(y_test, bayes_search.predict(X_test_processed)))
print("Best Params:", bayes_search.best_params_)


BayesSearchCV:
Train RMSE: 2.540485858246308
Test RMSE: 2.6232610267818806
Best Params: OrderedDict([('C', 4.699516044442124), ('epsilon', 1.0), ('kernel', 'rbf')])


# MLR

In [ ]:
# Train Multiple Linear Regression Model
mlr_model = LinearRegression()
mlr_model.fit(X_train_processed, y_train)

# Predictions on train and test data
y_train_pred = mlr_model.predict(X_train_processed)
y_test_pred = mlr_model.predict(X_test_processed)

# Calculate RMSE
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"Train RMSE: {train_rmse:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")

Train RMSE: 2.4554
Test RMSE: 2.7333


# Second-Order Multiple Linear Regression

In [ ]:
# second-order polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_processed)
X_test_poly = poly.transform(X_test_processed)

# Fit a Linear Regression model
model_poly = LinearRegression()
model_poly.fit(X_train_poly, y_train)

# Predict and evaluate
y_train_pred = model_poly.predict(X_train_poly)
y_test_pred = model_poly.predict(X_test_poly)

rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"Train RMSE: {rmse_train:.4f}")
print(f"Test RMSE: {rmse_test:.4f}")

Train RMSE: 0.0000
Test RMSE: 22.2491


# PCR

In [ ]:
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_processed)
X_test_pca = pca.transform(X_test_processed)

pc = LinearRegression()
pc.fit(X_train_pca, y_train)

# Step 3: Make predictions
y_train_pred = pc.predict(X_train_pca)
y_test_pred = pc.predict(X_test_pca)

# Step 4: Evaluate model performance (RMSE, R-squared)
train_rmse_pc = root_mean_squared_error(y_train, y_train_pred)
test_rmse_pc = root_mean_squared_error(y_test, y_test_pred)

train_r2_pc = r2_score(y_train, y_train_pred)
test_r2_pc = r2_score(y_test, y_test_pred)

# Output RMSE and R-squared for PCR
print(f"PCR Train RMSE: {train_rmse_pc:.4f}")
print(f"PCR Test RMSE: {test_rmse_pc:.4f}")
print(f"PCR Train R-squared: {train_r2_pc:.4f}")
print(f"PCR Test R-squared: {test_r2_pc:.4f}")

PCR Train RMSE: 2.6937
PCR Test RMSE: 2.6426
PCR Train R-squared: 0.1699
PCR Test R-squared: 0.1194


# END

In [ ]:
# Create a table of the data
table = {
    'Model': ['k-Nearest Neighbors (KNN)', 'Multilayer Perceptron Regressor', 'Random Forest', 'Support Vector Regressor', 'Multiple Linear Regression'],
    'Train RMSE': [2.8263, 2.5984, 1.9240, 2.430, 2.4554],
    'Test RMSE': [2.7235, 2.6090, 2.5552, 2.629, 2.7333],
    'Rank (Test RMSE)': [4, 2, 1, 3, 5]
}

df_results = pd.DataFrame(table)

df_results

,Model,Train RMSE,Test RMSE,Rank (Test RMSE)
0,k-Nearest Neighbors (KNN),2.8263,2.7235,4
1,Multilayer Perceptron Regressor,2.5984,2.6090,2
2,Random Forest,1.9240,2.5552,1
3,Support Vector Regressor,2.4300,2.6290,3
4,Multiple Linear Regression,2.4554,2.7333,5


There was a few interesting points in this dataset. There was an outlier in the target feature, 'Member years', but it was more like an error, since no one can be an member for -1806 years. So, I limited the member years to be nothing less then 0 years. Another data cleaning thing that I did was- one of the ordinal variables, 'Hotel stars', was being treated as a string because instead of the ordinal map being [3.5 4.  5.  3.  4.5] it was actually [3,5 4,  5,  3,  4,5]. So I changed ',' to '.' in the Hotel stars column.